## Update Views/Joined Views with SourceSchema to true


In [4]:
# Import the ArcGIS API for Python
from arcgis.gis import GIS
# Import the ArcGIS API for Python FeatureLayerCollection manager
from arcgis.features import FeatureLayerCollection
# Import requests, whiuch is another way to update feature layer definitions
import requests

# Connect to the current AGO organization. In ArcGIS Online notebooks, "home" will use the current credentials.
gis = GIS("home")

You are logged on as Mukwonago_Consultant with an administrator role, proceed with caution.


### Get a list of all views within the organization

In [5]:
# Use the seach method to get a list of views/joined views in the organization
all_views = gis.content.search(query='type:"Feature Service" AND typekeywords:"View Service"', max_items=1000)

### Set the ItemID of interest

In [9]:
# input the itemID of the HFL you want to adjust all views for
source_item_id = ""

### Initalize the empty itemID list of views to be updated

In [10]:
# Create an empty array
matching_view_ids = []

### Loop through all the views within the organization and determine what views need to be updated

In [11]:
# For every view
for view in all_views:
        try:
            # Create a ist of all items a view is related to (joined views may have two)
            related = view.related_items(rel_type="Service2Data")
            for item in related:
                # for every one of those items, check to see if it is the ID we specified
                if item.id == source_item_id:
                    matching_view_ids.append(view.id)
                    break  # No need to check /sources if already matched
        except Exception as e:
            print(f"Error checking view {view.id}: {e}")

### Print a list of all views that will be updated

In [12]:
print(f"\nViews that reference source item {source_item_id}:")
for vid in matching_view_ids:
    print(vid)


Views that reference source item 6a33f2387d584c9685678ed7ae84d21f:
44b6a8baa7b54c08ad9583d6e00c47ab


### Loop through the detected views and set source schema to true

In [13]:
# For every view
for vid in matching_view_ids:
    try:
        # Get the Item from the view ItemID
        view_item = gis.content.get(vid)
        # Convert to a FLC object
        flc = FeatureLayerCollection.fromitem(view_item)

        # Set the property of the view to true
        update_result = flc.manager.update_definition({
            "sourceSchemaChangesAllowed": True
        })

        print(f"Updated {vid}: {update_result}")

    except Exception as e:
        print(f"Failed to update {vid}: {e}")

Updated 44b6a8baa7b54c08ad9583d6e00c47ab: {'success': True}
